# Werken met TimeSeries
Soms hebben we reeksen die gebaseerd zijn op tijden. Een voorbeeld hiervan is het bestand 'full_grouped.csv' dat we al gedownload hebben. Maar hier volgt eerst de code om het opnieuw te doen.

In [1]:
from pathlib import Path
from zipfile import ZipFile
import requests
RAPPORT_ZIP = 'rapport.zip'
URL = 'https://www.kaggle.com/api/v1/datasets/download/imdevskp/corona-virus-report'
rapport_path = Path(RAPPORT_ZIP)
if not rapport_path.exists():
    data = requests.get(URL)

    with open(RAPPORT_ZIP, mode='wb') as f:
        f.write(data.content)

    with open(RAPPORT_ZIP, 'rb') as f:
        ZipFile(f).extractall()

Vervolgens kunnen we het bestand lezen in Pandas.

In [2]:
import pandas as pd
COVID_CSV = 'full_grouped.csv'
df = pd.read_csv(COVID_CSV, delimiter=',')
df.head()

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,WHO Region
0,2020-01-22,Afghanistan,0,0,0,0,0,0,0,Eastern Mediterranean
1,2020-01-22,Albania,0,0,0,0,0,0,0,Europe
2,2020-01-22,Algeria,0,0,0,0,0,0,0,Africa
3,2020-01-22,Andorra,0,0,0,0,0,0,0,Europe
4,2020-01-22,Angola,0,0,0,0,0,0,0,Africa


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35156 entries, 0 to 35155
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Date            35156 non-null  object
 1   Country/Region  35156 non-null  object
 2   Confirmed       35156 non-null  int64 
 3   Deaths          35156 non-null  int64 
 4   Recovered       35156 non-null  int64 
 5   Active          35156 non-null  int64 
 6   New cases       35156 non-null  int64 
 7   New deaths      35156 non-null  int64 
 8   New recovered   35156 non-null  int64 
 9   WHO Region      35156 non-null  object
dtypes: int64(7), object(3)
memory usage: 2.7+ MB


## parse_dates=[]
We zien hier dat de datum niet ingelezen is als een pandas tijd, maar als een object. We kunnen dit oplossen met het argument parse_dates. We zullen meteen ook Pandas strings gebruiken. 

In [4]:
df = pd.read_csv(COVID_CSV, delimiter=',', parse_dates=['Date'])
df = df.astype({'Country/Region':pd.StringDtype(), 'WHO Region':pd.StringDtype()})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35156 entries, 0 to 35155
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            35156 non-null  datetime64[ns]
 1   Country/Region  35156 non-null  string        
 2   Confirmed       35156 non-null  int64         
 3   Deaths          35156 non-null  int64         
 4   Recovered       35156 non-null  int64         
 5   Active          35156 non-null  int64         
 6   New cases       35156 non-null  int64         
 7   New deaths      35156 non-null  int64         
 8   New recovered   35156 non-null  int64         
 9   WHO Region      35156 non-null  string        
dtypes: datetime64[ns](1), int64(7), string(2)
memory usage: 2.7 MB


## We bekijken alleen de gegevens van België

In [ ]:
df_belgie = df.loc[df['Country/Region']=='Belgium'].copy() # om straks geen SettingWithCopyWarning te krijgen
df_belgie

## De .dt 'accessor'
We hebben al gezien dat we de string-functionaliteit kunnen aanspreken met .str. We hebben ook al gezien dat we de category functionaliteit kunnen aanspreken met .cat. Voor een Timestamp bestaat er ook zoiets: .dt. Bijvoorbeeld ik wil van elke datum het nummer van de weekdag zien.

In [ ]:

df_belgie.Date.dt.weekday

## Date als index
Misschien dat het een goed idee is om Date als index te gebruiken

In [ ]:
df_belgie.set_index('Date', inplace=True)
df_belgie.info()

## Datuminfo van een DatetimeIndex
Omdat dit een specifiek type is voor een Datetime van een Index, moeten we geen .dt gebruiken

In [ ]:
df_belgie['weekdag'] = df_belgie.index.weekday
df_belgie

## .to_period('W')
We kunnen nu op basis van de datum (index) een periode definiëren, bijvoorbeeld de week. Van elke record wordt er nu aangegeven tot welke week hij behoort.

In [ ]:

df_belgie = df.loc[df['Country/Region']=='Belgium'].copy() # Voorkomt de SettingWithCopyWarning
df_belgie = df_belgie.set_index('Date')
df_belgie['period'] = df_belgie.index.to_period('W')
df_belgie

## Groeperen per period
En nu kunnen we groeperen: totaal New cases voor elke week

In [ ]:
df_per_week = df_belgie.groupby('period')['New cases'].sum()
df_per_week

## In een grafiek

In [ ]:
df_per_week.plot()

## Een alternatief is de resample() functie
De resample()-functie werkt alleen met een Datetime Index. We kunnen de gegevens groeperen per week ('W') en vervolgens een aggregatiefunctie erop loslaten, zoals bijvoorbeel .sum()

In [ ]:
df_belgie = df[df['Country/Region']=='Belgium'].copy() # Voorkomt de SettingWithCopyWarning
df_belgie = df_belgie.set_index('Date')
df_per_week = df_belgie['New cases'].resample('W').sum()
df_per_week.plot()